# SAM3 物件追蹤與記憶機制

本筆記本演示如何利用 SAM3 的記憶機制進行強大的物件追蹤。我們將通過追蹤移動物體並使用新提示更新記憶庫來驗證記憶注意力模組的工作原理。


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import sys
from PIL import Image

# 確保 sam3 在路徑中
sys.path.append(os.getcwd())

from sam3.model.sam3_video_predictor import Sam3VideoPredictor
from sam3.visualization_utils import visualize_formatted_frame_output

# 配置
CHECKPOINT_PATH = os.path.abspath("sam3.pt")
VIDEO_DIR = os.path.abspath("tracking_demo_frames")

if os.path.exists(VIDEO_DIR):
    shutil.rmtree(VIDEO_DIR)
os.makedirs(VIDEO_DIR)

print(f"使用設備: {'mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')}")


In [ ]:
# 生成一個移動方塊的合成影片
# 方塊從左上角移動到右下角
# 在第 15 幀，它發生'跳躍'（模擬困難的運動或鏡頭切換）

FRAMES = 30
H, W = 400, 400

print(f"正在 {VIDEO_DIR} 中生成 {FRAMES} 幀...")

ground_truth_boxes = {}

for i in range(FRAMES):
    img = np.zeros((H, W, 3), dtype=np.uint8)
    
    # 線性運動
    step = 5
    x = 50 + i * step
    y = 50 + i * step
    
    # 模擬第 15 幀的跳躍/遮擋
    if i >= 15:
        x += 50 # 向右跳躍 50 像素
        y += 0
        
    w, h = 60, 60
    
    # 繪製方塊（紅色）
    cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), -1)
    
    # 保存幀
    fname = f"{i:05d}.jpg"
    cv2.imwrite(os.path.join(VIDEO_DIR, fname), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    
    ground_truth_boxes[i] = [x, y, x+w, y+h]

print("影片生成完成。")

# 可視化第 0 幀和第 15 幀
f0 = Image.open(os.path.join(VIDEO_DIR, "00000.jpg"))
f15 = Image.open(os.path.join(VIDEO_DIR, "00015.jpg"))

plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.imshow(f0); plt.title("第 0 幀")
plt.subplot(1,2,2); plt.imshow(f15); plt.title("第 15 幀 (跳躍)")
plt.show()


In [ ]:
# 初始化 SAM3 影片預測器
# 這將初始化記憶庫和影片骨幹網絡
if not os.path.exists(CHECKPOINT_PATH):
    print(f"在 {CHECKPOINT_PATH} 未找到檢查點")
else:
    predictor = Sam3VideoPredictor(checkpoint_path=CHECKPOINT_PATH)
    print("預測器初始化完成。")


In [ ]:
# 開始推理會話
# 讀取所有幀到記憶體中（或創建存取器）
session_response = predictor.handle_request({
    "type": "start_session",
    "resource_path": VIDEO_DIR
})
session_id = session_response["session_id"]
print(f"會話已開始: {session_id}")


In [ ]:
# 步驟 1：在第 0 幀使用方框提示初始化記憶
# 記憶編碼器將把這個遮罩壓縮成緊湊的記憶代碼。

x0, y0, x1, y1 = ground_truth_boxes[0]
w = x1 - x0
h = y1 - y0
# 參數通常期望 [x, y, w, h] 格式

prompt_box = [float(x0), float(y0), float(w), float(h)]
print(f"在第 0 幀提示方框: {prompt_box}")

response = predictor.handle_request({
    "type": "add_prompt",
    "session_id": session_id,
    "frame_index": 0,
    "bounding_boxes": [prompt_box],
    "obj_id": 1
})

# 可視化結果
# 顯示助手函數
def show_frame_result(frame_idx, output_dict):
    frame_path = os.path.join(VIDEO_DIR, f"{frame_idx:05d}.jpg")
    img = Image.open(frame_path)
    plt.figure(figsize=(6,6))
    plt.imshow(img)
    
    # 這裡可以繪製遮罩，sam3 輸出通常包含 'masks'
    plt.title(f"第 {frame_idx} 幀")
    plt.show()
    print(f"輸出鍵值: {response['outputs'][0].keys() if response['outputs'] else 'None'}")

show_frame_result(0, response["outputs"])


In [ ]:
# 步驟 2：在影片中傳播
# 模型使用"短期記憶"（最近的幀）和"長期記憶"（提示的幀）來追蹤物件。

tracked_results = {}
print("正在傳播...")

# propagate_in_video 生成每一幀的結果
propagation_generator = predictor.handle_stream_request({
    "type": "propagate_in_video",
    "session_id": session_id,
    "start_frame_index": 0,
    "propagation_direction": "forward"
})

for frame_res in propagation_generator:
    idx = frame_res["frame_index"]
    tracked_results[idx] = frame_res["outputs"]

print(f"已追蹤 {len(tracked_results)} 帧。")


In [ ]:
# 可視化第 16 幀（跳躍後）
# 如果跳躍太大，追蹤可能會漂移或失敗。
# 或者如果成功，則證明了強大的匹配能力。

vis_frame_idx = 16
# 我們可以可視化遮罩
try:
    if vis_frame_idx in tracked_results:
        print(f"第 {vis_frame_idx} 幀的結果: 發現 {len(tracked_results[vis_frame_idx])} 個物件")
        
        # 簡單顯示幀
        frame_path = os.path.join(VIDEO_DIR, f"{vis_frame_idx:05d}.jpg")
        plt.figure()
        plt.imshow(Image.open(frame_path))
        plt.title(f"第 {vis_frame_idx} 幀追蹤結果")
        plt.show()
        
    else:
        print(f"第 {vis_frame_idx} 幀無追蹤記錄")
except Exception as e:
    print(f"可視化錯誤: {e}")



In [ ]:
# 步驟 3：使用記憶更新改進追蹤
# 如果在跳躍處（第 15 幀）追蹤失敗或漂移，我們會手動提供一個新提示。
# 這種"交互"是 SAM 的核心優勢。它增加了一個新的記憶幀。
# 記憶庫現在有：[第 0 幀（記憶），第 15 幀（記憶）]

x_new, y_new, x1_new, y1_new = ground_truth_boxes[15]
w_new = x1_new - x_new
h_new = y1_new - y_new

print(f"在第 15 幀添加修正提示: {[x_new, y_new, w_new, h_new]}")

predictor.handle_request({
    "type": "add_prompt",
    "session_id": session_id,
    "frame_index": 15,
    "bounding_boxes": [[float(x_new), float(y_new), float(w_new), float(h_new)]],
    "obj_id": 1 # 使用相同的 ID 來更新特定物件的追蹤
})

# 從第 15 幀開始重新傳播
print("正在從第 15 幀開始重新傳播...")
propagation_generator = predictor.handle_stream_request({
    "type": "propagate_in_video",
    "session_id": session_id,
    "start_frame_index": 15,
    "propagation_direction": "forward"
})

for frame_res in propagation_generator:
    idx = frame_res["frame_index"]
    tracked_results[idx] = frame_res["outputs"]
    
print("更新追蹤完成。")
